In [ ]:
import matplotlib
matplotlib.use('Agg')
from matplotlib import pyplot as plt
from PIL import Image
import numpy as np
import torch
from transformers import GLPNImageProcessor, GLPNForDepthEstimation

In [ ]:
feature_extractor = GLPNImageProcessor.from_pretrained("vinvino02/glpn-nyu")
model = GLPNForDepthEstimation.from_pretrained("vinvino02/glpn-nyu")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
image = Image.open("/content/783848_Toyoto company in UK _xl-1024-v1-0.png").convert("RGB")
new_height = 480 if image.height > 480 else image.height
new_height = new_height - (new_height % 32)
new_width = int(image.width * new_height / image.height)
diff = new_width % 32

new_width = new_width - diff if diff < 16 else new_width + 32 - diff
new_size = (new_width, new_height)
image = image.resize(new_size)

In [ ]:
inputs = feature_extractor(images=image, return_tensors="pt")

In [ ]:
with torch.no_grad():
    outputs = model(**inputs)
    predicted_depth = outputs.predicted_depth

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

pad = 16
output = predicted_depth.squeeze().cpu().numpy() * 1000.0
output = output[pad:-pad, pad:-pad]
image = image.crop((pad, pad, image.width - pad, image.height - pad))

fig, ax = plt.subplots(1, 2)

ax[0].imshow(image)
ax[0].tick_params(
    bottom=False,
    labelbottom=False,
    left=False,
    labelleft=False,
)

ax[1].imshow(output, cmap="plasma")
ax[1].tick_params(
    bottom=False,
    labelbottom=False,
    left=False,
    labelleft=False,
)

plt.tight_layout()
plt.pause(5)

In [ ]:
import numpy as np
import open3d as o3d

In [ ]:
import numpy as np
import open3d as o3d

width, height = image.size
depth_image = (output * 255 / np.max(output)).astype("uint8")
image_np = np.array(image)

depth_o3d = o3d.geometry.Image(depth_image)
image_o3d = o3d.geometry.Image(np.array(image))
rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(
    image_o3d, depth_o3d, convert_rgb_to_intensity=False)

In [ ]:
camera_intrinsic = o3d.camera.PinholeCameraIntrinsic()
camera_intrinsic.set_intrinsics(width, height, 500, 500, width / 2, height / 2)

In [ ]:
pcd = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd_image, camera_intrinsic)
o3d.io.write_point_cloud("point_cloud.ply", pcd)

True

In [ ]:
cl, ind = pcd.remove_statistical_outlier(nb_neighbors=20, std_ratio=6.0)
pcd = pcd.select_by_index(ind)
pcd.estimate_normals()
pcd.orient_normals_to_align_with_direction()
o3d.io.write_point_cloud("cleaned_point_cloud.ply", pcd)

print("Point cloud saved as cleaned_point_cloud.ply")
pcd = o3d.io.read_point_cloud("/content/point_cloud.ply")

distance_threshold = 5.0

points = np.asarray(pcd.points)
distances = np.linalg.norm(points, axis=1)

indices = np.where(distances < distance_threshold)[0]
pcd = pcd.select_by_index(indices)

pcd.estimate_normals()
pcd.orient_normals_consistent_tangent_plane(30)
mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(
    pcd, o3d.utility.DoubleVector([0.05, 0.1, 0.2]))

o3d.io.write_triangle_mesh("output_mesh_without_background.ply", mesh)

o3d.visualization.draw_geometries([mesh])



Point cloud saved as cleaned_point_cloud.ply


In [ ]:
mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=10, n_threads=1)[0]
rotation = mesh.get_rotation_matrix_from_xyz((np.pi, 0, 0))
mesh.rotate(rotation, center=(0, 0, 0))

o3d.visualization.draw_geometries([mesh], mesh_show_back_face=True)

mesh_uniform = mesh.paint_uniform_color([0.9, 0.8, 0.9])
mesh_uniform.compute_vertex_normals()
o3d.visualization.draw_geometries([mesh_uniform], mesh_show_back_face=True)

[Open3D WARNING] GLFW Error: X11: The DISPLAY environment variable is missing
[Open3D WARNING] Failed to initialize GLFW
[Open3D WARNING] [DrawGeometries] Failed creating OpenGL window.
[Open3D WARNING] GLFW Error: X11: The DISPLAY environment variable is missing
[Open3D WARNING] Failed to initialize GLFW
[Open3D WARNING] [DrawGeometries] Failed creating OpenGL window.


In [ ]:
o3d.io.write_triangle_mesh("mesh.obj", mesh_uniform)